In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

Framework based interaction

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [ ]:
from context import Context

context = Context()

In [ ]:
from contracts.escrow_contract import EscrowContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract

contract: EscrowContract = context.get_contracts(config.ESCROWS)[0]
energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
locked_token = energy_contract.locked_token

upgrade contract

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
hash = contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.ESCROW_BYTECODE_PATH, [])

Lock tokens

In [ ]:
from utils.utils_chain import Account, WrapperAddress, nominated_amount
from utils.utils_tx import ESDTToken

user = Account(pem_file=config.DEFAULT_ADMIN)
receiver = Account(pem_file=config.DEFAULT_ADMIN)

user.address = WrapperAddress("erd1wfkgmzx36tjqurnvxjdu5pzmmyqg20p5k76mvp8sghsycngcaq5s5s0ecn")
receiver.address = WrapperAddress("erd1hfnw32gkydnj80cljjvkfz9kl3tachmwcpgjhxm5k5l0vhu4pr9s4zwxwa")
tokens = [ESDTToken(locked_token, int('09', 16), 6385888718117022425088000)]

user.sync_nonce(context.network_provider.proxy)
contract.lock_funds(user, context.network_provider.proxy, [tokens, receiver.address])

In [ ]:
user = Account(pem_file=config.DEFAULT_ADMIN)
receiver = Account(pem_file=config.DEFAULT_ADMIN)

user.address = WrapperAddress("erd1wfkgmzx36tjqurnvxjdu5pzmmyqg20p5k76mvp8sghsycngcaq5s5s0ecn")
receiver.address = WrapperAddress("erd1hfnw32gkydnj80cljjvkfz9kl3tachmwcpgjhxm5k5l0vhu4pr9s4zwxwa")

context.deployer_account.sync_nonce(context.network_provider.proxy)
contract.cancel_transfer(context.deployer_account, context.network_provider.proxy, [user.address, receiver.address])

Token searcher

In [ ]:
from utils.decoding_structures import XMEX_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes
from utils.utils_chain import base64_to_hex
from multiversx_sdk import ApiNetworkProvider

searched_offset = 30

# get current epoch
main_api = ApiNetworkProvider("https://api.multiversx.com")
current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number

for i in range(1, 10000):
    result = main_api.get_non_fungible_token(locked_token, i)
    if result:
        decoded_data = decode_merged_attributes(base64_to_hex(result.attributes), XMEX_ATTRIBUTES)
        print(i, decoded_data)
        if current_epoch < decoded_data['unlock_epoch'] < current_epoch + searched_offset:
            print(i)
            break
    

XMEX unlocking

In [ ]:
import time
from utils.utils_chain import get_all_token_nonces_details_for_account
from utils.utils_tx import ESDTToken

tokens = get_all_token_nonces_details_for_account(locked_token, user.address, context.network_provider.proxy)
print(len(tokens), tokens)
user.sync_nonce(context.network_provider.proxy)

token_list = []
for token in tokens:
    token_to_unlock = ESDTToken(locked_token, token["nonce"], int(token["balance"]))
    energy_contract.unlock_tokens(user, context.network_provider.proxy, [[token_to_unlock]])
    time.sleep(10)

In [ ]:
import time
from utils.utils_chain import get_all_token_nonces_details_for_account
from utils.utils_tx import ESDTToken

tokens = get_all_token_nonces_details_for_account(locked_token, user.address, context.network_provider.proxy)
print(len(tokens), tokens)
user.sync_nonce(context.network_provider.proxy)

token_list = []
for token in tokens:
    token_to_unlock = ESDTToken(locked_token, token["nonce"], int(token["balance"]))
    energy_contract.unlock_early(user, context.network_provider.proxy, [[token_to_unlock]])
    time.sleep(10)

In [ ]:
from contracts.pair_contract import PairContract
from contracts.dex_proxy_contract import DexProxyContract, DexProxyRemoveLiquidityEvent

pair_contract: PairContract = context.get_contracts(config.PAIRS_V2)[0]
dex_proxy_contract: DexProxyContract = context.get_contracts(config.PROXIES)[0]

print(dex_proxy_contract.proxy_lp_token)
tokens = get_all_token_nonces_details_for_account(dex_proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
for token in tokens:
    event = DexProxyRemoveLiquidityEvent(pair_contract, int(token["balance"]), token["nonce"], 1, 1)
    hash = dex_proxy_contract.remove_liquidity_proxy(user, context.network_provider.proxy, event)

In [ ]:
from contracts.locked_asset_contract import LockedAssetContract
from utils.utils_chain import get_all_token_nonces_details_for_account
from utils.utils_tx import ESDTToken
import time

locked_asset_contract: LockedAssetContract = context.get_contracts(config.LOCKED_ASSETS)[0]
tokens = get_all_token_nonces_details_for_account(locked_asset_contract.locked_asset, user.address.bech32(), context.network_provider.proxy)
for token in tokens:
    locked_asset_contract.unlock_assets(user, context.network_provider.proxy, [[ESDTToken(locked_asset_contract.locked_asset, token["nonce"], int(token["balance"]))]])
    time.sleep(10)

Low Level interaction

In [ ]:
from multiversx_sdk import Address, TokenPayment, ProxyNetworkProvider
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ESDTNFTTransferBuilder, DefaultTransactionBuildersConfiguration
from tools.notebooks import env

config = DefaultTransactionBuildersConfiguration(chain_id=env.CHAIN_ID)
proxy = ProxyNetworkProvider(env.PROXY_URL)

In [ ]:
contract_address = env.ESCROW_CONTRACT
XMEX = env.XMEX
LKMEX = env.LKMEX
WXMEX = env.WXMEX

In [ ]:
from multiversx_sdk import AccountNonceHolder, UserSigner
from pathlib import Path

user = env.USER1
receiver = env.USER2
admin = env.ADMIN
owner = env.OWNER

user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
owner_signer = UserSigner.from_pem_file(Path(env.OWNER_PEM))

lockFunds

In [ ]:
from multiversx_sdk import TransactionsConverter

user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_amount(XMEX, 1, 1000000, 18),
    TokenPayment.meta_esdt_from_amount(XMEX, 46, 1000000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="lockFunds",
    caller=user,
    call_arguments=[receiver],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()

converter = TransactionsConverter()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
from multiversx_sdk import TransactionComputer

tx.nonce = user_nonce_holder.get_nonce_then_increment()

tx_computer = TransactionComputer()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)

withdraw receiver

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(env.USER2).nonce)

builder = ContractCallBuilder(
    config,
    contract=env.ESCROW_CONTRACT,
    function_name="withdraw",
    caller=env.USER2,
    call_arguments=[env.FORWARDER_CONTRACT],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)

withdraw self

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="withdraw",
    caller=user,
    call_arguments=[user],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)

withdraw - others (fail)

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(admin).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="withdraw",
    caller=admin,
    call_arguments=[user],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)

cancel transfer - user (fail)

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="cancelTransfer",
    caller=user,
    call_arguments=[user, receiver],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)

addAdmin - owner (pass)

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(owner).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="addAdmin",
    caller=owner,
    call_arguments=[admin],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)

addAdmin - others (fail)

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="addAdmin",
    caller=user,
    call_arguments=[receiver],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)

cancel transfer

In [ ]:
user_nonce_holder = AccountNonceHolder(proxy.get_account(env.ADMIN).nonce)

builder = ContractCallBuilder(
    config,
    contract=env.ESCROW_CONTRACT,
    function_name="cancelTransfer",
    caller=admin,
    call_arguments=[env.USER1, env.FORWARDER_CONTRACT],
    gas_limit=30000000
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)

Token transfer

In [ ]:
transfer = TokenPayment.meta_esdt_from_amount(WXMEX, 2, 2000000, 18)

user = env.USER1
receiver = env.USER2
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ESDTNFTTransferBuilder(
    config,
    destination=receiver,
    sender=user,
    gas_limit=30000000,
    payment=transfer
)

tx = builder.build()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)